In [ ]:
#|default_exp callback.progresize

# Progressive Resizing
> Automatic progressive resizing of images during training

`ProgressiveResize` is inspired by MosaicML’s [Progressive Resizing algorithm for Composer](https://docs.mosaicml.com/en/stable/method_cards/progressive_resizing.html) which in turn was inspired by [fastai’s](https://github.com/fastai/fastbook/blob/780b76bef3127ce5b64f8230fce60e915a7e0735/07_sizing_and_tta.ipynb) manual progressive resizing.

![progressive resizing illustrated](images/progressive_resizing.png)

Progressive Resizing decreases model training time by training on smaller images then gradually increasing to the full image size. This allows training on more samples for the same compute budget, often leading to higher performance then training on full sized images.

In [ ]:
#|export
from __future__ import annotations

from pathlib import Path
from tempfile import TemporaryDirectory

from fastcore.basics import detuplify, in_notebook
from fastcore.transform import Pipeline

from fastai.callback.core import Callback
from fastai.callback.fp16 import MixedPrecision
from fastai.learner import _cast_tensor
from fastai.vision.augment import AffineCoordTfm, RandomResizedCropGPU

from fastxtend.imports import *

In [ ]:
#|exporti
_resize_augs = (AffineCoordTfm, RandomResizedCropGPU)

In [ ]:
#|exporti
def _to_size(t:Tensor):
    "Convert Tensor to size compatible values"
    if sum(t.shape)==2:
        return tuple(t.tolist())
    else:
        return tuple(t.item(),t.item())

In [ ]:
#|exporti
def _num_steps(final_size, current_size, increase_by):
    "Convert Tensor to size compatible values"
    steps = (final_size - current_size) / increase_by
    if sum(steps.shape)==2:
        steps = steps[0].item()
    return steps

In [ ]:
#|exporti
def _evenly_divisible(final_size, current_size, increase_by, steps):
    increase_by = tensor(increase_by)
    return (((final_size-current_size) % increase_by).sum() == 0) and (((final_size-current_size) - (increase_by*steps)).sum() == 0)

## ProgressiveResize -

In [ ]:
#|export
class IncreaseMode(Enum):
    "Increase mode for `ProgressiveResize`"
    Epoch = 'epoch'
    Batch = 'batch'

In [ ]:
#|export
class ProgressiveResize(Callback):
    "Progressively increase the size of input images during training. Starting from `initial_size` and ending at the valid image size or `final_size`."
    order = MixedPrecision.order+1 # Needs to run after MixedPrecision
    def __init__(self,
        initial_size:float|tuple[int,int]=0.5, # Staring size to increase from. Image shape must be square
        start:Numeric=0.5, # Earliest upsizing epoch in percent of training time or epoch (index 0)
        finish:Numeric=0.75, # Last upsizing epoch in percent of training time or epoch (index 0)
        increase_by:int=4, # Progressively increase image size by `increase_by`, or minimum increase per upsizing epoch
        increase_mode:IncreaseMode=IncreaseMode.Batch, # Increase image size by training percent or before an epoch starts
        resize_mode:str='bilinear', # PyTorch interpolate mode string for upsizing. Resets to existing fastai DataLoader mode at `final_size`.
        resize_valid:bool=True, # Apply progressive resizing to valid dataset
        final_size:tuple[int,int]|None=None, # Final image size. Set if using a non-fastai DataLoaders, automatically detected from fastai DataLoader with batch_tfms
        add_resize:bool=False, # Add a separate resize step. Use for non-fastai DataLoaders or fastai DataLoader without batch_tfms
        resize_targ:bool=False, # Applies the separate resize step to targets
        empty_cache:bool=False, # Call `torch.cuda.empty_cache()` before a resizing epoch. May prevent Cuda & Magma errors. Don't use with multiple GPUs
        verbose:bool=True, # Print a summary of the progressive resizing schedule
        logger_callback:str='wandb', # Log image size to `logger_callback` using `Callback.name` if available
    ):
        store_attr()
        self.run_valid = resize_valid
        if resize_targ and not add_resize:
            warn(f'`resize_targ` requires `add_resize` set to True')
        if empty_cache and increase_mode==IncreaseMode.Batch:
            warn(f'`empty_cache` requires `increase_mode` set to Epoch')

    def before_fit(self):
        "Sets up Progressive Resizing"
        if hasattr(self.learn, 'lr_finder') or hasattr(self.learn, "gather_preds"):
            self.run = False
            return

        self._resize, self._remove_resize, self._null_resize, self._remove_cutmix = [], False, True, False
        self._log_size = getattr(self, f'_{self.logger_callback}_log_size', noop)
        self.has_logger = hasattr(self.learn, self.logger_callback) and self._log_size != noop
        self.increase_by = tensor(self.increase_by)
        self.resize_batch = self.increase_mode == IncreaseMode.Batch

        # Dry run at full resolution to pre-allocate memory
        # See https://pytorch.org/tutorials/recipes/recipes/tuning_guide.html#pre-allocate-memory-in-case-of-variable-input-length
        states = get_random_states()
        path = self.path/self.model_dir
        path.mkdir(parents=True, exist_ok=True)
        tmp_d = TemporaryDirectory(dir=path)
        tmp_p = Path(tmp_d.name).stem
        self.learn.save(f'{tmp_p}/_tmp')
        try:
            b = self.dls.valid.one_batch()
            i = getattr(self.dls, 'n_inp', 1 if len(b)==1 else len(b)-1)
            self.learn.xb, self.learn.yb = b[:i],b[i:]

            if hasattr(self.learn, 'mixed_precision'):
                self.learn.mixed_precision.autocast.__enter__()

            self.learn.pred = self.learn.model(*_cast_tensor(self.learn.xb))
            self.learn.loss = self.learn.loss_func(self.learn.pred, *_cast_tensor(self.learn.yb))

            if hasattr(self.learn, 'mixed_precision'):
                self.learn.mixed_precision.autocast.__exit__(None, None, None)

            self.learn.loss.backward()
            self.learn.opt.zero_grad()

        finally:
            self.learn.load(f'{tmp_p}/_tmp', with_opt=True)
            tmp_d.cleanup()
            set_random_states(**states)

        # Try to automatically determine the input size
        try:
            for n in range(i):
                x = detuplify(self.learn.xb[n])
                if isinstance(x, TensorImageBase):
                    self.final_size = x.shape[-2:]
        finally:
            if self.final_size is None:
                raise ValueError(f'Could not determine image size from DataLoader. Set `final_size`: {self.final_size}')
            self.final_size = tensor(self.final_size)
            if self.final_size[0] != self.final_size[1]:
                raise ValueError(f'`ProgressiveResize` does not support non-square images: `final_size` = {self.final_size.tolist()}')
            if not self.resize_batch:
                if self.final_size[0] % 2 != 0:
                    raise ValueError(f"In Epoch mode, input image size must be even: {self.final_size.tolist()}")
                if self.increase_by.item() % 2 != 0:
                    raise ValueError(f"In Epoch Mode, `increase_by` must be even: {self.increase_by}")

        # Set the initial size
        if isinstance(self.initial_size, float):
            self.current_size = (tensor(self.initial_size) * self.final_size).int()
        elif isinstance(self.initial_size, tuple):
            self.current_size = tensor(self.initial_size)

        if self.resize_batch:
            # Set when the progressive resizing step is applied in training percent
            if self.start >= 1 and isinstance(self.start, int):
                self.start = self.start/self.n_epoch
            if self.finish >= 1 and isinstance(self.finish, int):
                self.finish = self.finish/self.n_epoch
            if self.start >= 1:
                warn(f'ProgressiveResize start {self.start} is equal or greater than one and will not start in this training run')
            if self.finish >= 1:
                warn(f'ProgressiveResize finish {self.finish} is equal or greater than one and will not finish in this training run')
            n_steps = ((self.final_size-self.current_size) / self.increase_by).int()
            if sum(n_steps.shape)==2:
                n_steps = n_steps[0].item()
            pct = (self.finish - self.start) / (n_steps-1)
            self.step_pcts = [self.start + pct*i for i in range(n_steps)]
        else:
            # Automatically determine the number of steps, increasing `increase_by` as needed
            start_epoch  = int(self.n_epoch*self.start)  if isinstance(self.start, float)  else self.start
            finish_epoch = int(self.n_epoch*self.finish) if isinstance(self.finish, float) else self.finish
            max_steps = finish_epoch - start_epoch
            count = 10000 # prevent infinite loop
            steps = _num_steps(self.final_size, self.current_size, self.increase_by)
            while ((steps > max_steps) or not _evenly_divisible(self.final_size, self.current_size, self.increase_by, steps)) and count > 0:
                self.increase_by += 2
                steps = _num_steps(self.final_size, self.current_size, self.increase_by)
                count -= 1
            n_steps = _num_steps(self.final_size, self.current_size, self.increase_by)

            # Set when per epoch progressive resizing steps are applied
            step_size = max(1, int(max_steps / n_steps))
            start_epoch = finish_epoch - ((self.final_size-self.current_size) / self.increase_by)*step_size
            if isinstance(start_epoch, torch.Tensor):
                if sum(start_epoch.shape)==2: start_epoch = int(start_epoch[0].item())
                else:                         start_epoch = int(start_epoch.item())
            self.step_epochs = [i for i in range(start_epoch+step_size, finish_epoch+step_size, step_size)]


        # Double check that the step size works
        if not _evenly_divisible(self.final_size, self.current_size, self.increase_by, n_steps):
            raise ValueError(f'Resize amount {self.final_size-self.current_size} not evenly divisible by `increase_by` {self.increase_by}')

        if self.verbose:
            if self.resize_batch:
                msg = f'Progressively increase the initial image size of {self.current_size.tolist()} by {self.increase_by} '\
                      f'pixels every {pct*self.n_epoch:.4g} epochs for {len(self.step_pcts)} resizes. \nStarting at epoch '\
                      f'{self.step_pcts[0]*self.n_epoch:.4g} and finishing at epoch {self.step_pcts[-1]*self.n_epoch:.4g} '\
                      f'for a final training size of {(self.current_size+(len(self.step_pcts))*self.increase_by).tolist()}.'
            else:
                msg = f'Progressively increase the initial image size of {self.current_size.tolist()} by {self.increase_by} '\
                      f'pixels every {step_size} epoch{"s" if step_size > 1 else ""} for {len(self.step_epochs)} resizes.\nStarting '\
                      f'at epoch {start_epoch+step_size} and finishing at epoch {finish_epoch} for a final training size of '\
                      f'{(self.current_size+(len(self.step_epochs))*self.increase_by).tolist()}.'
            print(msg) if in_notebook() else print('\n' + msg + '\n')

        # If not `add_resize`, check for fastai Augmentation resizes to use
        if not self.add_resize:
            if hasattr(self.learn, 'cut_mix_up_augment'):
                self._has_cutmixupaug = True
                # Modify the `CutMixUpAugment` augmentation pipeline
                self._process_pipeline(self.learn.cut_mix_up_augment._orig_pipe)

                # If `CutMixUpAugment` has an Affine Transform for Augmentations then use it
                if len(self._resize) > 0:
                    # Check for pre-mixup augment pipeline and modify it
                    if self.learn.cut_mix_up_augment._docutmixaug:
                        self._process_pipeline(self.learn.cut_mix_up_augment._cutmixaugs_pipe)
                        self.learn.cut_mix_up_augment._size = _to_size(self.current_size)
                    else:
                        # There isn't one, then add it a pre-mixup augment pipeline for resizing
                        self.learn.cut_mix_up_augment._cutmixaugs_pipe = Pipeline(AffineCoordTfm(size=_to_size(self.current_size)))
                        self.learn.cut_mix_up_augment._docutmixaug = True
                        self.learn.cut_mix_up_augment._size = _to_size(self.current_size)
                        self._resize.append(self.learn.cut_mix_up_augment._cutmixaugs_pipe[0])
                        self._remove_cutmix, self._remove_resize = True, True
            else:
                self._has_cutmixupaug = False
                # If no `CutMixUpAugment` check the train dataloader pipeline for Affine Transforms
                self._process_pipeline(self.dls.train.after_batch.fs)

            # If `resize_valid` check the valid dataloader pipeline for Affine Transforms
            if self.resize_valid:
                self._process_pipeline(self.dls.valid.after_batch.fs)

        # If `add_resize` or missing a fastai Augmentation resize add a seperate resize
        if self.add_resize or len(self._resize) == 0:
            self._added_resize = partial(F.interpolate, mode=self.resize_mode, recompute_scale_factor=True)
            self.add_resize, self._remove_resize = True, True

        # Set created or detected resize to the first size and store original interpolation
        self._orig_modes = []
        for resize in self._resize:
            resize.size = _to_size(self.current_size)
            self._orig_modes.append(resize.mode)
            resize.mode = self.resize_mode

    def before_batch(self):
        "Increases the image size before a batch if set to ProgSizeMode.Batch and applies optional additional resize"
        if self.training and self.resize_batch and len(self.step_pcts) > 0 and self.pct_train >= self.step_pcts[0]:
            _ = self.step_pcts.pop(0)
            self._increase_size()
        if self.add_resize:
            self.learn.xb = (self._added_resize(self.x, scale_factor=(self.current_size/self.final_size)[0]),)
            if self.resize_targ:
                self.learn.yb = (self._added_resize(self.y, scale_factor=(self.current_size/self.final_size)[0]),)

    def before_train(self):
        "Increases the image size before the training epoch if set to ProgSizeMode.Epoch"
        if self.epoch==0 and self.has_logger:
            self._log_size(False)

        if not self.resize_batch and len(self.step_epochs) > 0 and self.epoch >= self.step_epochs[0]:
            _ = self.step_epochs.pop(0)
            self._increase_size()

    def after_epoch(self):
        "Calls `torch.cuda.empty_cache()` if `empty_cache=True` before a resizing epoch if set to ProgSizeMode.Epoch. May slightly increase single GPU training speed."
        if not self.resize_batch and self.empty_cache and len(self.step_epochs) > 0 and self.epoch+1 >= self.step_epochs[0]:
            del self.learn.xb
            del self.learn.yb
            del self.learn.pred
            torch.cuda.empty_cache()

        if self.epoch+1==self.n_epoch and self.has_logger:
            self._log_size(False)

    def _increase_size(self):
        "Increase the input size"
        if self.has_logger:
            self._log_size(False)

        self.current_size += self.increase_by
        for i, resize in enumerate(self._resize):
            if (self.current_size < self.final_size).all():
                resize.size = _to_size(self.current_size)
                if self._has_cutmixupaug:
                    self.learn.cut_mix_up_augment._size = _to_size(self.current_size)
            else:
                # Reset everything after progressive resizing is done
                if self._null_resize:
                    resize.size = None
                    if self._has_cutmixupaug:
                        self.learn.cut_mix_up_augment._size = None
                else:
                    resize.size = _to_size(self.current_size)
                    resize.mode = self._orig_modes[i]

        if (self.current_size == self.final_size).all() and self._remove_resize:
                self.add_resize = False
                if self._remove_cutmix:
                    self.learn.cut_mix_up_augment._cutmixaugs_pipe = Pipeline([])
                    self.learn.cut_mix_up_augment._docutmixaug = False

        if self.has_logger:
            self._log_size()

    def _process_pipeline(self, pipe, null_resize=None):
        'Helper method for processing augmentation pipelines'
        for p in pipe:
            if isinstance(p, _resize_augs):
                self._resize.append(p)
                if null_resize is None:
                    self._null_resize = self._null_resize and p.size is None
                else:
                    self._null_resize = null_resize

Progressive Resizing initially trains on downsampled images then gradually increases the image size over to the full size for the remainder of training. 

This can significantly reduce training time at the possible expense of lower model performance. However, Progressive Resizing allows training on more samples within the same compute budget, usually leading to increased performance. 

The model must be capable of variable image sizes.

::: {.callout-important collapse='false'}
#### Important: Dataloader Bottleneck
<code>ProgressiveResize</code> should increase GPU throughput which may cause other parts of the training pipeline become a bottleneck.

An easy way to increase fastai’s DataLoader throughput is by [replacing Pillow with Pillow-SIMD](https://docs.fast.ai/dev/performance.html#pillow-simd).
:::

When testing Composer's [Progressive Resizing](https://docs.mosaicml.com/en/stable/method_cards/progressive_resizing.html) callback MosiacML [found]( https://docs.mosaicml.com/en/stable/method_cards/progressive_resizing.html#technical-details):

> In our experiments, Progressive Resizing improves the attainable tradeoffs between training speed and the final quality of the trained model. In some cases, it leads to slightly lower quality than the original model for the same number of training steps. However, Progressive Resizing increases training speed so much (via improved throughput during the early part of training) that it is possible to train for more steps, recover accuracy, and still complete training in less time.

`ProgressiveResize` modifies the fastai batch augmentation pipeline by changing the `batch_tfms` size during training. Specifically, it modifies `AffineCoordTfm` size, which is set by any rotate, warp, or resize batch augmentation, and/or `RandomResizedCropGPU` size. This modification prevents unnecessarily resizing images a second time on the GPU, speeding up the process. If there are no `batch_tfms` or if training with a non-fastai DataLoader, set `add_resize=True` to resize the batch on the GPU using PyTorch’s `interpolate`.

Progressive Resizing works best when the resize steps are spread out over a significant portion of the dataset.

::: {.callout-tip collapse='false'}
#### Tip: Progressive Resizing & Small Datasets
If training small datasets with <code>ProgressiveResize</code>, such as [Imagenette](https://github.com/fastai/imagenette), scale the batch mode increase amount to be larger than the default of 4 by setting `increase_by` to a custom value.

In the [example section](#example), `increase_by=16` gives good results for training Imagenette for 20-25 epochs.
:::

`ProgressiveResize` fully compatible with `CutMixUpAugment`.

::: {.callout-note collapse='true'}
#### Note: Older Versions of PyTorch
If training on older versions of PyTorch with <code>ProgressiveResize</code> results in CUDA or Magma errors, try setting <code>increase_mode=IncreaseMode.Epoch</code> and <code>empty_cache=True</code>.

This will upsize once per epoch and call <code>torch.cuda.empty_cache()</code> before a resizing epoch. <code>empty_cache=True</code> may interfere with training multiple models on multi-GPU systems.
:::

## Tests -

In [ ]:
#|hide
from fastxtend.test_utils import *

In [ ]:
#|hide
class SyncthProgResizeTest(Callback):
    order = ProgressiveResize.order+1
    def __init__(self, final_size, start_size, increase, step_size, first_epoch, last_epoch, total_resizes):
        store_attr()

    def before_fit(self):
        prog = self.learn.progressive_resize
        if isinstance(self.start_size, tuple):
            assert torch.equal(prog.current_size, tensor(self.start_size).int())
        else:
            assert torch.equal(prog.current_size, tensor([self.start_size,self.start_size]).int())
        assert prog.increase_by==self.increase
        assert prog.step_epochs[1]-prog.step_epochs[0]==self.step_size
        assert prog.step_epochs[0]==self.first_epoch
        assert prog.step_epochs[-1]==self.last_epoch
        assert len(prog.step_epochs)==self.total_resizes
        size = self.start_size
        for i in prog.step_epochs:
            size += self.increase
        assert size==self.final_size

In [ ]:
#|hide
test = SyncthProgResizeTest(final_size=256, start_size=128, increase=32, step_size=1, first_epoch=12, last_epoch=15, total_resizes=4)
learn = synth_learner(cbs=[ProgressiveResize(final_size=[256,256], increase_mode=IncreaseMode.Epoch), test])
learn('after_create')
learn.create_opt()
learn.n_epoch=20
learn('before_fit')

Progressively increase the initial image size of [128, 128] by 32 pixels every 1 epoch for 4 resizes.
Starting at epoch 12 and finishing at epoch 15 for a final training size of [256, 256].


In [ ]:
#|hide
test = SyncthProgResizeTest(final_size=256, start_size=128, increase=8, step_size=1, first_epoch=34, last_epoch=49, total_resizes=16)
learn = synth_learner(cbs=[ProgressiveResize(final_size=[256,256], increase_mode=IncreaseMode.Epoch), test])
learn('after_create')
learn.create_opt()
learn.n_epoch=66
learn('before_fit')

Progressively increase the initial image size of [128, 128] by 8 pixels every 1 epoch for 16 resizes.
Starting at epoch 34 and finishing at epoch 49 for a final training size of [256, 256].


In [ ]:
#|hide
test = SyncthProgResizeTest(final_size=384, start_size=192, increase=6, step_size=2, first_epoch=163, last_epoch=225, total_resizes=32)
learn = synth_learner(cbs=[ProgressiveResize(final_size=[384,384], increase_by=6, increase_mode=IncreaseMode.Epoch), test])
learn('after_create')
learn.create_opt()
learn.n_epoch=300
learn('before_fit')

Progressively increase the initial image size of [192, 192] by 6 pixels every 2 epochs for 32 resizes.
Starting at epoch 163 and finishing at epoch 225 for a final training size of [384, 384].


In [ ]:
#|hide
test = SyncthProgResizeTest(final_size=384, start_size=192, increase=8, step_size=3, first_epoch=156, last_epoch=225, total_resizes=24)
learn = synth_learner(cbs=[ProgressiveResize(final_size=[384,384], increase_by=8, increase_mode=IncreaseMode.Epoch), test])
learn('after_create')
learn.create_opt()
learn.n_epoch=300
learn('before_fit')

Progressively increase the initial image size of [192, 192] by 8 pixels every 3 epochs for 24 resizes.
Starting at epoch 156 and finishing at epoch 225 for a final training size of [384, 384].


In [ ]:
#|hide
learn = synth_learner(cbs=[ProgressiveResize(final_size=[256,256], increase_by=4, increase_mode=IncreaseMode.Batch)])
learn('after_create')
learn.create_opt()
learn.n_epoch=80
learn('before_fit')

Progressively increase the initial image size of [128, 128] by 4 pixels every 0.6452 epochs for 32 resizes. 
Starting at epoch 40 and finishing at epoch 60 for a final training size of [256, 256].


In [ ]:
#|hide
learn = synth_learner(cbs=[ProgressiveResize(final_size=[384,384], increase_by=4, increase_mode=IncreaseMode.Batch)])
learn('after_create')
learn.create_opt()
learn.n_epoch=300
learn('before_fit')

Progressively increase the initial image size of [192, 192] by 4 pixels every 1.596 epochs for 48 resizes. 
Starting at epoch 150 and finishing at epoch 225 for a final training size of [384, 384].


In [ ]:
#|hide
#|slow
import time
from packaging.version import parse
import fastai

from fastcore.basics import num_cpus

if parse(fastai.__version__) < parse('2.7.11'):
    from fastxtend.callback.channelslast import *
else:
    from fastai.callback.channelslast import *
from fastai.data.external import URLs, untar_data
from fastai.data.block import DataBlock, CategoryBlock
from fastai.data.transforms import GrandparentSplitter, get_image_files, parent_label, Normalize
from fastai.learner import Learner
from fastai.vision.augment import Resize, aug_transforms
from fastai.vision.core import imagenet_stats
from fastai.vision.data import ImageBlock
from fastai.vision.models import resnet50, resnet34
from fastxtend.callback.cutmixup import CutMixUpAugment
from fastxtend.metrics import *
from fastxtend.optimizer.fused import adam, ranger
from fastxtend.vision.models.xresnet import xresnext50
from fastxtend.utils import *

In [ ]:
#|hide
#|slow
free_gpu_memory(learn)

In [ ]:
#|hide
#|slow
class ProgressiveResizeTest(Callback):
    run_valid, order = True, ProgressiveResize.order-1

    def before_train(self):
        self.progsize = self.learn.progressive_resize.current_size

    def before_batch(self):
        assert L(self.x.shape[-2:]) == L(self.progsize.tolist())

    def after_batch(self):
        self.progsize = self.learn.progressive_resize.current_size

In [ ]:
#|hide
#|slow
#|cuda
imagenette = untar_data(URLs.IMAGENETTE_160)

with less_random():
    dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                        splitter=GrandparentSplitter(valid_name='val'),
                        get_items=get_image_files, get_y=parent_label,
                        item_tfms=Resize(128),
                        batch_tfms=[*aug_transforms(), Normalize.from_stats(*imagenet_stats)])
    dls = dblock.dataloaders(imagenette, bs=128, num_workers=num_cpus(), pin_memory=True)

    cbs=[ProgressiveResize(increase_by=16), ProgressiveResizeTest]
    learn = Learner(dls, resnet34(num_classes=dls.c), opt_func=adam(foreach=True), cbs=cbs).to_channelslast()
    learn.fit_one_cycle(1, 3e-3)
    free_gpu_memory(learn, dls)

Progressively increase the initial image size of [64, 64] by 16 pixels every 0.08333 epochs for 4 resizes. 
Starting at epoch 0.5 and finishing at epoch 0.75 for a final training size of [128, 128].


epoch,train_loss,valid_loss,time
0,1.809523,1.559901,00:07


In [ ]:
#|hide
#|slow
#|cuda
imagenette = untar_data(URLs.IMAGENETTE_160)

with less_random():
    dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                        splitter=GrandparentSplitter(valid_name='val'),
                        get_items=get_image_files, get_y=parent_label,
                        item_tfms=Resize(128),
                        batch_tfms=[*aug_transforms(), Normalize.from_stats(*imagenet_stats)])
    dls = dblock.dataloaders(imagenette, bs=128, num_workers=num_cpus(), pin_memory=True)

    cbs=[ProgressiveResize(increase_mode=IncreaseMode.Epoch), ProgressiveResizeTest]
    learn = Learner(dls, resnet34(num_classes=dls.c), opt_func=adam(foreach=True), cbs=cbs).to_channelslast()
    learn.fit_one_cycle(3, 3e-3)
    free_gpu_memory(learn, dls)

Progressively increase the initial image size of [64, 64] by 64 pixels every 1 epoch for 1 resizes.
Starting at epoch 2 and finishing at epoch 2 for a final training size of [128, 128].


epoch,train_loss,valid_loss,time
0,1.933080,1.900782,00:06
1,1.563433,1.386105,00:06
2,1.221522,1.025261,00:06


In [ ]:
#|hide
#|slow
#|cuda
imagenette = untar_data(URLs.IMAGENETTE_160)

with less_random():
    dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                        splitter=GrandparentSplitter(valid_name='val'),
                        get_items=get_image_files, get_y=parent_label,
                        item_tfms=Resize(128),
                        batch_tfms=[*aug_transforms(), Normalize.from_stats(*imagenet_stats)])
    dls = dblock.dataloaders(imagenette, bs=128, num_workers=num_cpus(), pin_memory=True)

    cbs=[ProgressiveResize(increase_by=16), ProgressiveResizeTest, CutMixUpAugment]
    learn = Learner(dls, resnet34(num_classes=dls.c), opt_func=adam(foreach=True), cbs=cbs).to_channelslast()
    learn.fine_tune(1, 3e-3)
    free_gpu_memory(learn, dls)

Progressively increase the initial image size of [64, 64] by 16 pixels every 0.08333 epochs for 4 resizes. 
Starting at epoch 0.5 and finishing at epoch 0.75 for a final training size of [128, 128].


epoch,train_loss,valid_loss,time
0,2.347264,3.078582,00:06


Progressively increase the initial image size of [64, 64] by 16 pixels every 0.08333 epochs for 4 resizes. 
Starting at epoch 0.5 and finishing at epoch 0.75 for a final training size of [128, 128].


epoch,train_loss,valid_loss,time
0,1.803510,1.596348,00:06


In [ ]:
#|hide
#|slow
#|cuda
imagenette = untar_data(URLs.IMAGENETTE_160)

with less_random():
    dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                        splitter=GrandparentSplitter(valid_name='val'),
                        get_items=get_image_files, get_y=parent_label,
                        item_tfms=Resize(128),
                        batch_tfms=[*aug_transforms(), Normalize.from_stats(*imagenet_stats)])
    dls = dblock.dataloaders(imagenette, bs=128, num_workers=num_cpus(), pin_memory=True)

    cbs=[ProgressiveResize(increase_by=16), ProgressiveResizeTest,
         CutMixUpAugment(cutmixup_augs=aug_transforms(max_rotate=45))]
    learn = Learner(dls, resnet34(num_classes=dls.c), opt_func=adam(foreach=True), cbs=cbs).to_channelslast()
    learn.fine_tune(1, 3e-3)
    free_gpu_memory(learn, dls)

Progressively increase the initial image size of [64, 64] by 16 pixels every 0.08333 epochs for 4 resizes. 
Starting at epoch 0.5 and finishing at epoch 0.75 for a final training size of [128, 128].


epoch,train_loss,valid_loss,time
0,2.346271,3.759643,00:07


Progressively increase the initial image size of [64, 64] by 16 pixels every 0.08333 epochs for 4 resizes. 
Starting at epoch 0.5 and finishing at epoch 0.75 for a final training size of [128, 128].


epoch,train_loss,valid_loss,time
0,1.806576,1.609734,00:06


## Example

In this example[^details], a `xresnext50` is trained for 20 & 25 epochs on [Imagenette](https://github.com/fastai/imagenette) at an image size of 224 pixels. Due to the short training run and small dataset, `ProgressiveResize` in batch mode is set to `increase_by=16`.

<code>ProgressiveResize</code> yields significant training time savings compared to training at full size. At a normalized compute budget of roughly 6.5 minutes, Progressive Resizing results with 92.7% accuracy compared to 92% accuracy with full sized training.

| Mode              | Epochs | Time (Mins) | Accuracy |
|:----------------- |:------:|:-----------:|:--------:|
| Full Size         | 20     | 6.5         | 92.0%    |
| Progressive Batch | 20     | 5.2         | 92.3%    |
| Progressive Epoch | 20     | 5.2         | 91.8%    |
| Progressive Batch | 25     | 6.5         | 92.7%    |

Due to the regularization effect of training on different sized images, Progressive Resizing with `increase_by=16` outperforms full sized training by 0.3% in 25 percent less timeon the same number of epochs[^result] .

[^details]: All models are trained on a GeForce 3080 Ti using PyTorch 1.13.1 and Cuda 11.7. Results may differ with other datasets, hardware, and across runs.

[^result]: While Progressive Resizing can sometimes outperform full sized trained model in the same number of epochs, it is just as likely to perform worse, depending on setup.

In [ ]:
#|hide
#|slow
free_gpu_memory(learn)

### Progressive Resizing

There are two Progressive Resizing `IncreaseMode`:

- `increase_mode=IncreaseMode.Batch`
- `increase_mode=IncreaseMode.Epoch`

this example will show both.

#### Batch Resizing

`ProgressiveResize` with the default `increase_mode=IncreaseMode.Batch`.

In [ ]:
#|slow
#|cuda
imagenette = untar_data(URLs.IMAGENETTE_320)

with less_random():
    dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                       splitter=GrandparentSplitter(valid_name='val'),
                       get_items=get_image_files, get_y=parent_label,
                       item_tfms=Resize(224),
                       batch_tfms=[*aug_transforms(), Normalize.from_stats(*imagenet_stats)])
    dls =  dblock.dataloaders(imagenette, bs=64, num_workers=num_cpus(), pin_memory=True)

    learn = Learner(dls, xresnext50(n_out=dls.c), loss_func=nn.CrossEntropyLoss(label_smoothing=0.1),
                    opt_func=ranger(foreach=True), metrics=Accuracy()).to_channelslast()

    start = time.perf_counter()
    learn.fit_flat_cos(20, 8e-3, cbs=ProgressiveResize(increase_by=16))
    total = time.perf_counter() - start
    print(f'Total training time: {scale_time(total)}')

Progressively increase the initial image size of [112, 112] by 16 pixels every 0.8333 epochs for 7 resizes. 
Starting at epoch 10 and finishing at epoch 15 for a final training size of [224, 224].


epoch,train_loss,valid_loss,accuracy,time
0,1.670977,1.883999,0.454268,00:13
1,1.403678,1.226364,0.710573,00:13
2,1.251599,1.446574,0.626497,00:13
3,1.136825,1.079901,0.768662,00:13
4,1.062239,1.250891,0.718981,00:13
5,1.006945,0.955187,0.820127,00:13
6,0.957047,1.238453,0.703439,00:13
7,0.910177,0.900485,0.842548,00:13
8,0.889880,0.963289,0.816560,00:13
9,0.860453,0.881689,0.849936,00:13


Total training time: 311.8 s


In [ ]:
#|hide
#|slow
#|cuda
free_gpu_memory(learn, dls)

#### Epoch Resizing

`ProgressiveResize` with `increase_mode=IncreaseMode.Epoch`.

In [ ]:
#|slow
#|cuda
imagenette = untar_data(URLs.IMAGENETTE_320)

with less_random():
    dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                        splitter=GrandparentSplitter(valid_name='val'),
                        get_items=get_image_files, get_y=parent_label,
                        item_tfms=Resize(224),
                        batch_tfms=[*aug_transforms(), Normalize.from_stats(*imagenet_stats)])
    dls =  dblock.dataloaders(imagenette, bs=64, num_workers=num_cpus(), pin_memory=True)

    learn = Learner(dls, xresnext50(n_out=dls.c), loss_func=nn.CrossEntropyLoss(label_smoothing=0.1),
                    opt_func=ranger(foreach=True), metrics=Accuracy()).to_channelslast()

    start = time.perf_counter()
    learn.fit_flat_cos(20, 8e-3, cbs=ProgressiveResize(increase_mode=IncreaseMode.Epoch))
    total = time.perf_counter() - start
    print(f'Total training time: {scale_time(total)}')

Progressively increase the initial image size of [112, 112] by 28 pixels every 1 epoch for 4 resizes.
Starting at epoch 12 and finishing at epoch 15 for a final training size of [224, 224].


epoch,train_loss,valid_loss,accuracy,time
0,1.670977,1.883999,0.454268,00:13
1,1.403678,1.226364,0.710573,00:13
2,1.251599,1.446574,0.626497,00:13
3,1.136825,1.079901,0.768662,00:13
4,1.062239,1.250891,0.718981,00:13
5,1.006945,0.955187,0.820127,00:13
6,0.957047,1.238453,0.703439,00:13
7,0.910177,0.900485,0.842548,00:13
8,0.889880,0.963289,0.816560,00:13
9,0.860453,0.881689,0.849936,00:14


Total training time: 309.3 s


In [ ]:
#|hide
#|slow
#|cuda
free_gpu_memory(learn, dls)

### Normal Training

fastai model training without Progressive Resizing.

In [ ]:
#|slow
#|cuda
imagenette = untar_data(URLs.IMAGENETTE_320)

with less_random():
    dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                       splitter=GrandparentSplitter(valid_name='val'),
                       get_items=get_image_files, get_y=parent_label,
                       item_tfms=Resize(224),
                       batch_tfms=[*aug_transforms(), Normalize.from_stats(*imagenet_stats)])
    dls =  dblock.dataloaders(imagenette, bs=64, num_workers=num_cpus(), pin_memory=True)

    learn = Learner(dls, xresnext50(n_out=dls.c), loss_func=nn.CrossEntropyLoss(label_smoothing=0.1),
                    opt_func=ranger(foreach=True), metrics=Accuracy()).to_channelslast()

    start = time.perf_counter()
    learn.fit_flat_cos(20, 8e-3)
    total = time.perf_counter() - start
    print(f'Total training time: {scale_time(total)}')

epoch,train_loss,valid_loss,accuracy,time
0,1.693837,1.660484,0.539873,00:19
1,1.425402,1.288508,0.682548,00:19
2,1.249855,1.231204,0.726879,00:19
3,1.107746,1.027718,0.794904,00:19
4,1.046856,1.113385,0.782420,00:19
5,0.974740,1.055205,0.800255,00:19
6,0.933220,1.195850,0.756688,00:19
7,0.880307,0.905752,0.845096,00:19
8,0.854195,1.113956,0.772229,00:19
9,0.839854,0.838828,0.868790,00:19


Total training time: 387.2 s


In [ ]:
#|hide
#|slow
#|cuda
free_gpu_memory(learn, dls)

### Progressive Resizing with Normalized Compute Budget

`ProgressiveResize` with the default `increase_mode=IncreaseMode.Batch` trained to match the [Normal Training's](#normal-training) compute.

In [ ]:
#|slow
#|cuda
imagenette = untar_data(URLs.IMAGENETTE_320)

with less_random():
    dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                       splitter=GrandparentSplitter(valid_name='val'),
                       get_items=get_image_files, get_y=parent_label,
                       item_tfms=Resize(224),
                       batch_tfms=[*aug_transforms(), Normalize.from_stats(*imagenet_stats)])
    dls =  dblock.dataloaders(imagenette, bs=64, num_workers=num_cpus(), pin_memory=True)

    learn = Learner(dls, xresnext50(n_out=dls.c), loss_func=nn.CrossEntropyLoss(label_smoothing=0.1),
                    opt_func=ranger(foreach=True), metrics=Accuracy()).to_channelslast()

    start = time.perf_counter()
    learn.fit_flat_cos(25, 8e-3, cbs=ProgressiveResize(increase_by=16))
    total = time.perf_counter() - start
    print(f'Total training time: {scale_time(total)}')

Progressively increase the initial image size of [112, 112] by 16 pixels every 1.042 epochs for 7 resizes. 
Starting at epoch 12.5 and finishing at epoch 18.75 for a final training size of [224, 224].


epoch,train_loss,valid_loss,accuracy,time
0,1.670977,1.883999,0.454268,00:13
1,1.403678,1.226364,0.710573,00:13
2,1.251599,1.446574,0.626497,00:13
3,1.136825,1.079901,0.768662,00:13
4,1.062239,1.250891,0.718981,00:13
5,1.006945,0.955187,0.820127,00:13
6,0.957047,1.238453,0.703439,00:13
7,0.910177,0.900485,0.842548,00:13
8,0.889880,0.963289,0.816560,00:13
9,0.860453,0.881689,0.849936,00:13


Total training time: 390.5 s


## Weights & Biases Logging

If Weights & Biases is installed and the [`WandbCallback`](https://docs.fast.ai/callback.wandb.html) is added to `Learner`, Progressive Resizing will automatically log the current image size to Weights & Biases as `progressive_resize_size`.

## Extend to other Loggers

To extend to new loggers, follow the Weights & Biases code below and create patches for `ProgressiveResize` to add a `_{Callback.name}_log_size`, where `Callback.name` is the [name of the logger callback](https://docs.fast.ai/callback.core.html#Callback.name).

In [ ]:
#|exports
try:
    import wandb

    @patch
    def _wandb_log_size(self:ProgressiveResize, next_step=True):
        size = _to_size(self.current_size)
        wandb.log({'progressive_resize_size': size[0]}, self.learn.wandb._wandb_step+int(next_step))
except:
    pass

Then to use, pass `logger_callback='{Callback.name}'` to `ProgressiveResize`.

`ProgressiveResize` sets its `_log_size` method to `f'_{self.logger_callback}_log_size'`, which should match the patched method.

```python
self._log_size = getattr(self, f'_{self.logger_callback}_log_size', noop)
```